!pip install scipy --quiet
!pip install tenacity --quiet
#!pip install tiktoken==0.3.3 --quiet
!pip install termcolor --quiet
!pip install openai --quiet
!pip install arxiv --quiet
!pip install pandas --quiet
!pip install PyPDF2 --quiet
!pip install tqdm --quiet

In [1]:
import os
import ast
import concurrent
import json
import os
import pandas as pd
import tiktoken
from IPython.display import display, Markdown, Latex
from openai import OpenAI
from PyPDF2 import PdfReader
from tenacity import retry, wait_random_exponential, stop_after_attempt

GPT_MODEL = "gpt-4-turbo"
EMBEDDING_MODEL = "text-embedding-ada-002"
client = OpenAI()

In [2]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    response = client.embeddings.create(input=text, model=EMBEDDING_MODEL)
    return response

## Configure Agent

We'll create our agent in this step, including a ```Conversation``` class to support multiple turns with the API, and some Python functions to enable interaction between the ```ChatCompletion``` API and our knowledge base functions.

In [3]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            functions=functions,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [4]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [5]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    full_message = response.choices[0]
    if full_message.finish_reason == "function_call":
        print(f"Function generation requested, calling function")
        return call_analog_function(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response

In [6]:
analog_design_functions = [
    {
        "name": "solve_analog_question",
        "description": """Use this function to answer user questions related to analog design, such as filter design, amplification, and circuit behavior.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": """
                        The user's query regarding analog design. Responses should focus on explaining key principles, 
                        providing equations, and outlining design choices (e.g., for filters or amplifiers).
                    """,
                }
            },
            "required": ["query"],
        },
    },
    {
        "name": "design_and_summarize",
        "description": """Use this function to go through specific steps to design analog components, 
                        such as calculating values for a high-pass filter. This should only be called after 
                        solve_analog_question has provided the basic design principles.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": """
                        Detailed description of the design task, such as 'Design a high-pass filter with a cutoff frequency of 1 kHz'. 
                        This should result in a step-by-step design summary with component choices and calculations.
                    """,
                },
                "specifications": {
                    "type": "object",
                    "properties": {
                        "cutoff_frequency": {
                            "type": "number",
                            "description": "The cutoff frequency for the filter in Hz."
                        },
                        "filter_type": {
                            "type": "string",
                            "description": "Type of filter to design, such as 'RC' or 'LC'."
                        },
                        "desired_gain": {
                            "type": "number",
                            "description": "Desired gain level, if applicable (for active filters)."
                        },
                    },
                    "required": ["cutoff_frequency", "filter_type"]
                },
            },
            "required": ["query", "specifications"],
        },
    },
    {
        "name": "calculate_pole_frequency",
        "description": """Calculate the pole (cutoff) frequency of a circuit based on resistance and capacitance values. This function is commonly used 
                        to determine the cutoff point for filters.""",
        "parameters": {
            "type": "object",
            "properties": {
                "resistance": {
                    "type": "number",
                    "description": "The resistance in ohms (Ω) for the circuit."
                },
                "capacitance": {
                    "type": "number",
                    "description": "The capacitance in farads (F) for the circuit."
                }
            },
            "required": ["resistance", "capacitance"],
        },
    },
    {
        "name": "two_port_amplifier_gain",
        "description": """Calculate the output voltage and gain of a two-port amplifier model. This is useful for analyzing amplifier circuits 
                        and determining the output for given input and circuit parameters.""",
        "parameters": {
            "type": "object",
            "properties": {
                "Rin": {
                    "type": "number",
                    "description": "Input resistance in ohms (Ω)."
                },
                "A": {
                    "type": "number",
                    "description": "Amplifier gain, typically dimensionless."
                },
                "Rout": {
                    "type": "number",
                    "description": "Output resistance in ohms (Ω)."
                },
                "Vin": {
                    "type": "number",
                    "description": "Input voltage in volts (V)."
                }
            },
            "required": ["Rin", "A", "Rout", "Vin"],
        },
    },
    {
        "name": "sctc_octc_analysis",
        "description": """Determine if a capacitor should be treated as an open or short circuit for SCTC (Short-Circuit Time Constant) or OCTC (Open-Circuit Time Constant) analysis 
                        based on its type in the circuit.""",
        "parameters": {
            "type": "object",
            "properties": {
                "capacitor_type": {
                    "type": "string",
                    "description": "Type of capacitor usage, specify 'high-pass' or 'low-pass'."
                }
            },
            "required": ["capacitor_type"],
        },
    }
]


In [7]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    full_message = response.choices[0]
    if full_message.finish_reason == "function_call":
        print(f"Function generation requested, calling function")
        return call_analog_function(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response


def call_analog_function(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""

    function_name = full_message.message.function_call.name
    try:
        # Parse the arguments from JSON format
        parsed_output = json.loads(full_message.message.function_call.arguments)
    except json.JSONDecodeError as e:
        print("Failed to parse function arguments.")
        print(f"Error message: {e}")
        raise

    results = None  # Initialize results

    # Handle different functions based on the function name
    if function_name == "solve_analog_question":
        try:
            print("Solving analog question")
            results = solve_analog_question(parsed_output["query"])
        except Exception as e:
            print(f"Function execution failed for solve_analog_question.")
            print(f"Error message: {e}")

    elif function_name == "design_and_summarize":
        try:
            print("Designing and summarizing")
            results = design_and_summarize(
                parsed_output["query"],
                parsed_output["specifications"]
            )
        except Exception as e:
            print(f"Function execution failed for design_and_summarize.")
            print(f"Error message: {e}")

    elif function_name == "calculate_pole_frequency":
        try:
            print("Calculating pole frequency")
            results = calculate_pole_frequency(
                parsed_output["resistance"],
                parsed_output["capacitance"]
            )
        except Exception as e:
            print(f"Function execution failed for calculate_pole_frequency.")
            print(f"Error message: {e}")

    elif function_name == "two_port_amplifier_gain":
        try:
            print("Calculating two-port amplifier gain")
            results = two_port_amplifier_gain(
                parsed_output["Rin"],
                parsed_output["A"],
                parsed_output["Rout"],
                parsed_output["Vin"]
            )
        except Exception as e:
            print(f"Function execution failed for two_port_amplifier_gain.")
            print(f"Error message: {e}")

    elif function_name == "sctc_octc_analysis":
        try:
            print("Performing SCTC/OCTC analysis")
            results = sctc_octc_analysis(parsed_output["capacitor_type"])
        except Exception as e:
            print(f"Function execution failed for sctc_octc_analysis.")
            print(f"Error message: {e}")

    else:
        raise Exception(f"Function {function_name} does not exist and cannot be called")

    # Append the results as a message in the conversation
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": str(results),
        }
    )

    # Generate the response using the results
    try:
        print(f"Got results for {function_name}, generating response")
        response = chat_completion_request(messages)
        return response
    except Exception as e:
        print("Function chat request failed.")
        print(f"Error message: {e}")
        raise


def solve_analog_question(query):
    """Handles general analog questions by explaining principles and considerations."""
    # Sample response - in a real implementation, this would interact with a model or knowledge base
    return f"To design a high-pass filter, you must select a filter type (e.g., RC, LC). Key considerations include the cutoff frequency, which determines how the filter attenuates lower frequencies, and the choice of components based on desired impedance."

def design_high_pass_filter(query, specifications):
    """Provides a detailed design for a high-pass filter based on specifications."""
    # Extract specifications
    cutoff_frequency = specifications["cutoff_frequency"]
    filter_type = specifications["filter_type"]
    # Example design calculation for an RC high-pass filter
    if filter_type.lower() == "rc":
        r_value = 1e3  # Assuming R = 1k ohms
        c_value = 1 / (2 * 3.14159 * cutoff_frequency * r_value)
        return f"For a {cutoff_frequency} Hz RC high-pass filter: \n - Resistor (R): {r_value} ohms \n - Capacitor (C): {c_value:.6f} F"

def calculate_pole_frequency(resistance, capacitance):
    """
    Calculate the pole frequency (cutoff frequency) for a low-pass or high-pass filter circuit.
    
    Parameters:
    - resistance (float): The resistance value in ohms (Ω).
    - capacitance (float): The capacitance value in farads (F).
    
    Returns:
    - float: The pole frequency (ω_p) in radians per second (rad/s).
    """
    tau = resistance * capacitance  # Time constant (τ)
    omega_p = 1 / tau  # Pole frequency
    return omega_p

def two_port_amplifier_gain(Rin, A, Rout, Vin):
    """
    Calculate the output voltage and gain of a two-port amplifier circuit model.
    
    Parameters:
    - Rin (float): Input resistance in ohms (Ω).
    - A (float): Amplifier gain (dimensionless).
    - Rout (float): Output resistance in ohms (Ω).
    - Vin (float): Input voltage in volts (V).
    
    Returns:
    - tuple (float, float): Output voltage (Vout) and gain (A).
    """
    Vout = A * Vin * (Rin / (Rin + Rout))
    return Vout, A

def sctc_octc_analysis(capacitor_type):
    """
    Determine if a capacitor should be treated as an open or short circuit for SCTC (Short-Circuit Time Constant)
    or OCTC (Open-Circuit Time Constant) analysis based on its role in the circuit.
    
    Parameters:
    - capacitor_type (str): The type of filter application, either 'high-pass' or 'low-pass'.
    
    Returns:
    - str: 'Open Circuit' or 'Short Circuit' based on the SCTC/OCTC rules.
    """
    if capacitor_type == "high-pass":
        return "Short Circuit"
    elif capacitor_type == "low-pass":
        return "Open Circuit"
    else:
        return "Invalid type"


In [8]:
# Start with a system message
paper_system_message = """You are a useful analog electronics tutor. Use the content and functions provided to guide students to ideate, think critically and solve analog electronincs problems in a fast and effective way?"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)

# Add a user message
paper_conversation.add_message("user", "Hi, how to design a low-pass filter for 200Hz?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=analog_design_functions
)

assistant_message = chat_response.choices[0].message.content
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

Function generation requested, calling function
Solving analog question
Got results for solve_analog_question, generating response


To design a low-pass filter with a cutoff frequency of 200 Hz, you can use a simple RC (Resistor-Capacitor) filter. This type of filter is easy to design and understand. Here's how to go about it:

### Step 1: Choose the Filter Type
We'll use an RC low-pass filter, which consists of a resistor and a capacitor.

### Step 2: Determine the Cutoff Frequency Formula
The cutoff frequency \( f_c \) for an RC low-pass filter is given by the formula:
\[
f_c = \frac{1}{2\pi RC}
\]
where:
- \( R \) is the resistance in ohms.
- \( C \) is the capacitance in farads.
- \( f_c \) is the cutoff frequency in hertz (200 Hz in this case).

### Step 3: Choose Values for R or C
You can choose a convenient value for either the resistor or the capacitor, and then calculate the other. Common practice is to start with a standard capacitor value. Let's start with a capacitor value:
- Let \( C = 1 \, \mu F \) (or \( 1 \times 10^{-6} \) farads).

### Step 4: Calculate Resistor Value
Plug the values into the cutoff frequency formula and solve for \( R \):
\[
200 = \frac{1}{2\pi R \times 1 \times 10^{-6}}
\]
Rearrange the formula to solve for \( R \):
\[
R = \frac{1}{2\pi \times 200 \times 1 \times 10^{-6}} \approx 796.18 \, \Omega
\]
You can use a standard resistor value close to this calculated value, such as 820 Ω or 780 Ω, depending on availability.

### Step 5: Build the Circuit
Connect the resistor and capacitor in series with the input signal connected across the resistor, and the output taken from the junction between the resistor and capacitor. Here's a simple diagram:
```
Input ----R----|---- Output
               |
               C
               |
             Ground
```
### Step 6: Verify the Design
You can verify the design by simulating the circuit in a software tool like LTSpice, or build it on a breadboard and measure the output with an oscilloscope or a frequency analyzer.

This simple design approach gives you a basic RC low-pass filter. If you require a steeper roll-off or specific performance criteria, you might consider more complex designs like Butterworth, Chebyshev, or active filters using operational amplifiers.

In [9]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    "Calculate the output voltage and gain for a two-port amplifier circuit with an input resistance of 1000 ohms, an output resistance of 500 ohms, an amplifier gain of 20, and an input voltage of 5 volts. I need to know the resulting output voltage",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=analog_design_functions
)
display(Markdown(updated_response.choices[0].message.content))


Function generation requested, calling function
Calculating two-port amplifier gain
Got results for two_port_amplifier_gain, generating response


To find the output voltage and gain in a two-port amplifier circuit given the parameters, let’s break down the calculation into two parts.

1. **Determine the gain under load conditions**: The input resistance and output resistance affect how the amplifier gain (voltage gain) is applied to the load. The actual load that sees this gain includes both the output resistance of the amplifier and the resistance of any load connected to it.

2. **Calculate the output voltage**: Using the derived gain and the input voltage, calculate the final output voltage across the load.

Given:
- Input resistance (\( R_{in} \)) = 1000 ohms
- Output resistance (\( R_{out} \)) = 500 ohms
- Amplifier gain (\( A_v \)) = 20
- Input voltage (\( V_{in} \)) = 5 volts

### Step 1: Calculate Load Gain
Considering no external load mentioned, the gain calculation remains straightforward:
\[ A_v' = A_v \frac{R_{out}}{R_{in} + R_{out}} \]
\[ A_v' = 20 \times \frac{500}{1000 + 500} \]
\[ A_v' = 20 \times \frac{500}{1500} \]
\[ A_v' = 20 \times \frac{1}{3} \]
\[ A_v' = \frac{20}{3} \approx 6.67 \]

### Step 2: Find the Output Voltage 
The output voltage (\( V_{out} \)) is calculated by multiplying the load gain by the input voltage:
\[ V_{out} = A_v' \times V_{in} \]
\[ V_{out} = \frac{20}{3} \times 5 \]
\[ V_{out} = \frac{100}{3} \approx 33.33 \text{ volts} \]

**Conclusion**:
- The output voltage across the load (or at the output terminals if not connected to any external load other than the inherent output resistance) is approximately 33.33 volts.
- The effective gain is approximately 6.67 under the given load (considering the output resistance as the load).